<a href="https://colab.research.google.com/github/rakshajp/steel_construction_manual_tables/blob/main/steel_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [215]:
!pip install PyPDF2

In [216]:
import PyPDF2
import csv

aisc_table_data.pdf file has been loaded in the file section of google colab notebook and accessed form there.

It can be found in the repo here:

pdf with 89 pages each page in a complex structured table

To run code on it, please download it form the repo and add it to colab notebook/feel free to write a function so iut can be accessed via the repository. (TO DO)

In [237]:
pdf_path = "/content/aisc_table_data.pdf"

In [218]:
def read_pdf():
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        extracted_text = ""
        extracted_pages = []
        for page_number in range(1, num_pages + 1):
            if(page_number%10==0):
              print('finished reading pages till page: ',page_number)
            page = pdf_reader.pages[page_number - 1]
            page_text = page.extract_text()
            extracted_text += f"Page {page_number}:\n{page_text}\n\n"
            extracted_pages.append(page_text)
        print('Done extracting pages from pdf: ',pdf_path)
        return extracted_pages,extracted_text

In [219]:
def numOfDesignUnitsPairs(str, designUnit='p×103'):
  return str.count(designUnit)

def findUniqueDesignValues(str):
  all_values=str.split(' ')
  return list(set(all_values))

In [220]:
def find_table_start_idx(string_list,substring='ShapeW'):
    occurrences = 0
    for idx, string in enumerate(string_list):
        if substring in string:
            occurrences += 1
            if occurrences == 1:
                return idx
    return -1

In [221]:
def process_and_split(input_string):
    # Replace non-digit characters with spaces
    replaced_string = ''.join(char if char.isdigit() else ' ' for char in input_string)

    stripped_string = replaced_string.strip()  # Remove extra spaces at the start and end
    split_list = stripped_string.split()        # Split the string into a list
    return split_list

In [222]:
extracted_pdf_data = read_pdf()
all_pages = extracted_pdf_data[0]
all_pages_str = extracted_pdf_data[1]

finished reading pages till page:  10
finished reading pages till page:  20
finished reading pages till page:  30
finished reading pages till page:  40
finished reading pages till page:  50
finished reading pages till page:  60
finished reading pages till page:  70
finished reading pages till page:  80
Done extracting pages from pdf:  /content/aisc_table_data.pdf


In [223]:
# print(all_pages[1]) #where number is basically num of page - 1

In [224]:
print('ShapeW ',all_pages_str.count('ShapeW'))
print('Shapes ',all_pages_str.count('Shapes'))

ShapeW  89
Shapes  89


In [225]:
count_of_ambiguous_pages=0
ambi_pages=[]

shape_column = []
kl_lb_ft = []
all_4_column_data = []
delt=[]

for page_number in range(len(all_pages)):
  page_data = all_pages[page_number]

  # shape_freq = str(page_data).count('Shape')

  page_lines = page_data.strip().split("\n")

  table_start_idx = find_table_start_idx(page_lines)
  shape_line = page_lines[table_start_idx]

  shape_values=shape_line.replace('Shape','').split(' ')

  if(len(shape_values)!=1):  #edge case
    # confused which pair is under which shape
    # hard - code for now?
    # print('weird: ', shape_values, '\n')
    # print(page_data)
    count_of_ambiguous_pages+=1
    ambi_pages.append(page_number+1)

  else:
    # numOfB usually = numOfP
    num_of_p = numOfDesignUnitsPairs(page_lines[table_start_idx+2],'p×103')
    num_of_b = numOfDesignUnitsPairs(page_lines[table_start_idx+2],'bx×103')

    # wont be using much
    design_units = page_lines[3]
    value_pairs = findUniqueDesignValues(page_lines[table_start_idx+4])  #usually just ['ASD', 'LRFD']

    # print('page: ', page_number+1)

    cleaned_string = process_and_split(page_lines[table_start_idx+1])

    if(len(cleaned_string)==num_of_b):
      shape_head_dict={shape_values[0]:cleaned_string}
      # print(shape_head_dict)

      sh = []

      for key, values in shape_head_dict.items():
          new_values = [f"{key}{value}" for value in values]
          sh.extend(new_values)

      # print(sh)
      shape_column.append(sh)


      table_values_data = [line.split() for line in page_lines[table_start_idx+5:]]

      # Find the index where the table ends (first non-number row)
      curr_table_end_index = len(table_values_data)
      for i, row in enumerate(table_values_data):
          if not row[0].isdigit():
              curr_table_end_index = i
              break

      # Trim rows to only include the table data
      table_values_data = table_values_data[:curr_table_end_index]
      kl_lb_ft.append([sublist[0] for sublist in table_values_data])

      # print()

      print(table_values_data)
      for i in table_values_data:
        x=[]
        for j in range(num_of_b):
          # print(i[(j*4)+1:((j+1)*4)+1])
          delt.append(i[(j*4)+1:((j+1)*4)+1])
          x.append(i[(j*4)+1:((j+1)*4)+1])
        all_4_column_data.append(x)


    else:
      # yay NONE!
      print('hopefully you never see this')

print('\nNum of pages not processed due to ambiguouity',count_of_ambiguous_pages, ambi_pages)

print('\n',shape_column,'\n',kl_lb_ft,'\n',all_4_column_data,'\n',delt)

print('\n',len(shape_column),'\n',len(kl_lb_ft),'\n',len(all_4_column_data),'\n',len(delt))

[['0', '0.346', '0.230', '0.220', '0.146', '0.417', '0.278', '0.253', '0.168', '0.474', '0.316', '0.281', '0.187'], ['11', '0.378', '0.251', '0.220', '0.146', '0.454', '0.302', '0.253', '0.168', '0.516', '0.343', '0.281', '0.187'], ['12', '0.384', '0.256', '0.220', '0.146', '0.462', '0.307', '0.253', '0.168', '0.524', '0.349', '0.281', '0.187'], ['13', '0.392', '0.261', '0.222', '0.148', '0.470', '0.313', '0.255', '0.170', '0.533', '0.355', '0.284', '0.189'], ['14', '0.402', '0.267', '0.225', '0.150', '0.480', '0.319', '0.259', '0.173', '0.544', '0.362', '0.289', '0.192'], ['15', '0.412', '0.274', '0.229', '0.152', '0.490', '0.326', '0.264', '0.175', '0.555', '0.369', '0.294', '0.196'], ['16', '0.423', '0.281', '0.232', '0.155', '0.501', '0.333', '0.268', '0.178', '0.568', '0.378', '0.299', '0.199'], ['17', '0.435', '0.290', '0.236', '0.157', '0.514', '0.342', '0.273', '0.181', '0.582', '0.387', '0.304', '0.203'], ['18', '0.449', '0.299', '0.240', '0.160', '0.527', '0.351', '0.277', '0

In [226]:

shape_and_design = []

for item1, sublist2 in zip(shape_column, kl_lb_ft):
    for item2 in sublist2:
        for item in item1:
            shape_and_design.append([item,str(item2)])


print(len(shape_and_design),len(delt))



5723 5723


In [227]:
for i in range(len(delt)):
  print(shape_and_design[i],delt[i])

Streaming output truncated to the last 5000 lines.
['W36×441', '24'] ['0.381', '0.254', '0.206', '0.137']
['W36×395', '24'] ['0.431', '0.287', '0.234', '0.155']
['W36×361', '24'] ['0.474', '0.316', '0.260', '0.173']
['W36×441', '26'] ['0.408', '0.272', '0.211', '0.140']
['W36×395', '26'] ['0.462', '0.307', '0.239', '0.159']
['W36×361', '26'] ['0.509', '0.339', '0.267', '0.178']
['W36×441', '28'] ['0.440', '0.293', '0.215', '0.143']
['W36×395', '28'] ['0.498', '0.331', '0.245', '0.163']
['W36×361', '28'] ['0.550', '0.366', '0.274', '0.183']
['W36×441', '30'] ['0.476', '0.317', '0.220', '0.147']
['W36×395', '30'] ['0.540', '0.359', '0.251', '0.167']
['W36×361', '30'] ['0.597', '0.397', '0.282', '0.188']
['W36×441', '32'] ['0.518', '0.345', '0.225', '0.150']
['W36×395', '32'] ['0.589', '0.392', '0.258', '0.172']
['W36×361', '32'] ['0.652', '0.434', '0.290', '0.193']
['W36×441', '34'] ['0.567', '0.377', '0.231', '0.153']
['W36×395', '34'] ['0.646', '0.430', '0.265', '0.176']
['W36×361', '3

In [228]:
# for page in range(len(shape_column)):
#   for shape in range(len(shape_column[page])):
#     for design in range(len(kl_lb_ft[page])):

#       print(shape_column[page][shape]+'_'+str(kl_lb_ft[page][design])+'_'+str(all_4_column_data[design*(shape+1)][shape]))



In [229]:
print(len(all_4_column_data),len(delt))

1932 5723


In [230]:


count=0
t=0
temp_total=0
for page in range(len(shape_column)):

  temp=(len(shape_column[page]))*(len(kl_lb_ft[page]))
  t+=len(kl_lb_ft[page])
  temp_total+=temp
  # print(page,t,len(kl_lb_ft[page]),temp,temp_total)

  for shape in range(len(shape_column[page])):
    for design in range(len(kl_lb_ft[page])):
      # print(shape_column[page][shape]+'_'+str(kl_lb_ft[page][design])+'_'+str(all_4_column_data[shape*(len(kl_lb_ft[page]))][shape]))
      # print(shape_column[page][shape]+'_'+str(kl_lb_ft[page][design])+'_'+str(delt[(len(shape_column[page])*(len(kl_lb_ft[page])))-1]))

      print(count,shape_column[page][shape]+'_'+str(kl_lb_ft[page][design])+'_'+str(shape_and_design[count]))
      count+=1




Streaming output truncated to the last 5000 lines.
723 W36×395_20_['W36×441', '24']
724 W36×395_22_['W36×395', '24']
725 W36×395_24_['W36×361', '24']
726 W36×395_26_['W36×441', '26']
727 W36×395_28_['W36×395', '26']
728 W36×395_30_['W36×361', '26']
729 W36×395_32_['W36×441', '28']
730 W36×395_34_['W36×395', '28']
731 W36×395_36_['W36×361', '28']
732 W36×395_38_['W36×441', '30']
733 W36×395_40_['W36×395', '30']
734 W36×395_42_['W36×361', '30']
735 W36×395_44_['W36×441', '32']
736 W36×395_46_['W36×395', '32']
737 W36×395_48_['W36×361', '32']
738 W36×395_50_['W36×441', '34']
739 W36×361_0_['W36×395', '34']
740 W36×361_11_['W36×361', '34']
741 W36×361_12_['W36×441', '36']
742 W36×361_13_['W36×395', '36']
743 W36×361_14_['W36×361', '36']
744 W36×361_15_['W36×441', '38']
745 W36×361_16_['W36×395', '38']
746 W36×361_17_['W36×361', '38']
747 W36×361_18_['W36×441', '40']
748 W36×361_19_['W36×395', '40']
749 W36×361_20_['W36×361', '40']
750 W36×361_22_['W36×441', '42']
751 W36×361_24_['W36×395',

In [231]:
# Only thing left:

csv_file = "aisc_table_data.csv"

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["shape", "kl_lb_ft", "p_asd", "p_lrfd", "bx_asd", "bx_lrfd"])  # Header

    for i in range(len(delt)):
      row = shape_and_design[i] + delt[i]
      # print(','.join(row))
      writer.writerow(row)
      print(row)

print(f"Data has been written to ${csv_file}")


Streaming output truncated to the last 5000 lines.
['W36×395', '24', '0.431', '0.287', '0.234', '0.155']
['W36×361', '24', '0.474', '0.316', '0.260', '0.173']
['W36×441', '26', '0.408', '0.272', '0.211', '0.140']
['W36×395', '26', '0.462', '0.307', '0.239', '0.159']
['W36×361', '26', '0.509', '0.339', '0.267', '0.178']
['W36×441', '28', '0.440', '0.293', '0.215', '0.143']
['W36×395', '28', '0.498', '0.331', '0.245', '0.163']
['W36×361', '28', '0.550', '0.366', '0.274', '0.183']
['W36×441', '30', '0.476', '0.317', '0.220', '0.147']
['W36×395', '30', '0.540', '0.359', '0.251', '0.167']
['W36×361', '30', '0.597', '0.397', '0.282', '0.188']
['W36×441', '32', '0.518', '0.345', '0.225', '0.150']
['W36×395', '32', '0.589', '0.392', '0.258', '0.172']
['W36×361', '32', '0.652', '0.434', '0.290', '0.193']
['W36×441', '34', '0.567', '0.377', '0.231', '0.153']
['W36×395', '34', '0.646', '0.430', '0.265', '0.176']
['W36×361', '34', '0.716', '0.477', '0.299', '0.199']
['W36×441', '36', '0.624', '0.4

Random calc/code for Manual Data Aggregation for pages : [2, 21, 26, 32, 37, 39, 43, 45, 52, 68]

In [232]:
# with open("test.csv", mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["shape", "kl_lb_ft", "p_asd", "p_lrfd", "bx_asd", "bx_lrfd"])  # Header

#     for i in range(len(delt)):
#       row = shape_and_design[i] + delt[i]
#       # print(','.join(row))
#       writer.writerow(row)
#       print(row)

# print(f"Data has been written to test")






In [233]:
# for i in range(len(delt)):
#   print(shape_and_design[i]+delt[i])

In [234]:
print(len(all_4_column_data))
# for i,idx in enumerate(all_4_column_data):
#   print(i,len(idx))

1932


In [235]:
print(len(shape_column))
print(ambi_pages)

79
[2, 21, 26, 32, 37, 39, 43, 45, 52, 68]


In [236]:

for idx,i in enumerate(shape_column):
  if(len(i)!=3):
    # ambi_pages.append(idx-1) cant do this you dunno where the missing block is
    print(idx,len(i))

46 2
67 2
75 2
76 2
